In [45]:
import pandas as pd
from functools import reduce
import xlsxwriter
import pprint

#### 1. Read the financial inclusion file
##### a. Navigate to the right working dir
##### b. Read all the sheets/tabs of the excel file

In [46]:
import os
os.getcwd()
folder = '/Users/anshulsaxena/MicroSave/cohort 3/contingency plan/Data Sources/wip'
os.chdir(folder)
file_name = input("Please enter the file you wish to process") 
#file_name = 'World Bank data 7 July MSC R4 countries.xlsx' # This file has been taken from the World Bank site

path = folder+'/'+file_name
df_raw_fin_data = []

df_read_all_sheets = pd.read_excel(path, sheet_name=None) # None value for sheet_name reads all sheets; 
#If there is an index column, we are handling it later in the code



read_sheet = input(f"Enter the sheet name to be read from the workbook {file_name}: ")

df_raw_fin_data = df_read_all_sheets[read_sheet]
df_raw_fin_data.head()


Please enter the file you wish to process Doing_Business_World_Bank_Data.xlsx
Enter the sheet name to be read from the workbook Doing_Business_World_Bank_Data.xlsx:  Data


,Unnamed: 0,Country Name,Country Code,Series Name,Series Code,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,0,Bangladesh,BGD,Enforcing contracts: Time (days),ENF.CONT.DURS.DY,..,..,..,..,..,..,..,..,..,..
1,1,Bangladesh,BGD,Global: Ease of doing business score (DB10-14 ...,IC.BUS.EASE.DFRN.DB1014,..,..,..,46.2484,..,..,..,..,..,..
2,2,Bangladesh,BGD,Paying taxes: Profit tax (% of profits),PAY.TAX.PRFT.CP.ZS,..,..,..,31.6,31.6,30.5,30.5,31.1,31.1,31.1
3,3,Bangladesh,BGD,Paying taxes: Time (hours per year),PAY.TAX.TM,..,..,..,422,422,422,435,435,435,435
4,4,Bangladesh,BGD,Paying taxes: Total tax and contribution rate ...,PAY.TAX.TOT.TAX.RT.ZS,..,..,..,35.4,35.4,34.4,34.4,33.4,33.4,33.4


#### 2. Tidying the data
##### World Bank data is to be converted from wide to long format esp. for tableau viz
##### 2a. Drop unnecessary columns

In [48]:
print(f"Original column names\n{df_raw_fin_data.columns}")

#if Unnamed:0 column exists, drop it
if 'Unnamed: 0' in df_raw_fin_data.columns:
    print ("dropping the unnamed col")
    df_raw_fin_data.drop(columns='Unnamed: 0', inplace=True)
else:
    print("No Unnamed: 0 column exists, we are good to go")

#print(df_raw_fin_data.info)
rename_to_country = input(f"Enter the column to be renamed to country in {file_name} sheet {read_sheet}: ")
rename_to_country_code = input(f"Enter the column to be renamed to code in {file_name} sheet {read_sheet}: ")

df_raw_fin_data.rename(columns={rename_to_country : "Country Name", rename_to_country_code : "Country Code"}, inplace=True)

print(f"Columns renamed to Country Name and Country Code; New column names\n {df_raw_fin_data.columns}\n")

print(df_raw_fin_data.head())



Original column names
Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]',
       '2019 [YR2019]', '2020 [YR2020]'],
      dtype='object')
No Unnamed: 0 column exists, we are good to go


Enter the column to be renamed to country in Doing_Business_World_Bank_Data.xlsx sheet Data:  Country Name
Enter the column to be renamed to code in Doing_Business_World_Bank_Data.xlsx sheet Data:  Country Code


Columns renamed to Country Name and Country Code; New column names
 Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]',
       '2019 [YR2019]', '2020 [YR2020]'],
      dtype='object')

  Country Name Country Code  \
0   Bangladesh          BGD   
1   Bangladesh          BGD   
2   Bangladesh          BGD   
3   Bangladesh          BGD   
4   Bangladesh          BGD   

                                         Series Name              Series Code  \
0                   Enforcing contracts: Time (days)         ENF.CONT.DURS.DY   
1  Global: Ease of doing business score (DB10-14 ...  IC.BUS.EASE.DFRN.DB1014   
2            Paying taxes: Profit tax (% of profits)       PAY.TAX.PRFT.CP.ZS   
3                Paying taxes: Time (hours per year)               PAY.TAX.TM   
4  Paying taxes: Total tax and contribution rate ...    

##### 2b. Compress the different 'year' columns into a single 'year' column

In [50]:
selected_cols = ['Country Name', 'Series Code', 'Series Name']

unselected_cols = ['Country Code']

df_fin_data_long_format = pd.melt(df_raw_fin_data.drop(columns=unselected_cols) # melt() converts wide to long format
                               , id_vars=selected_cols, var_name= 'year'\
                               , value_name = 'value')
                                
#rename columns

df_fin_data_long_format.rename(columns={
    "Country Name" : "country",
    "Series Name" : "indicator", "Series Code" : "code"
}, inplace=True)   



# tidy up the year column to show in 'yyyy' format
df_fin_data_long_format['year'] = df_fin_data_long_format['year'].str[0:4]

df_fin_data_long_format.head(100)

,country,code,indicator,year,value
0,Bangladesh,ENF.CONT.DURS.DY,Enforcing contracts: Time (days),2011,..
1,Bangladesh,IC.BUS.EASE.DFRN.DB1014,Global: Ease of doing business score (DB10-14 ...,2011,..
2,Bangladesh,PAY.TAX.PRFT.CP.ZS,Paying taxes: Profit tax (% of profits),2011,..
3,Bangladesh,PAY.TAX.TM,Paying taxes: Time (hours per year),2011,..
4,Bangladesh,PAY.TAX.TOT.TAX.RT.ZS,Paying taxes: Total tax and contribution rate ...,2011,..
...,...,...,...,...,...
95,Indonesia,PROT.MINOR.INV.IC.PRIN.EXT.DIR.LGL.010.XD,Protecting minority investors: Extent of direc...,2011,5
96,Indonesia,PAY.TAX.TOT.TAX.RT.ZS.DRFRN,Paying taxes: Total tax and contribution rate ...,2011,..
97,Indonesia,PROT.MINOR.INV.DFRN.DB0614,Protecting minority investors (DB06-14 methodo...,2011,60
98,Indonesia,IC.BUS.EASE.XQ,Rank: Ease of doing business index (1=most bus...,2011,..


#### 3. Writing the long formatted data back to the file
##### We'll write the new long formatted table back to the excel file
##### Since ExcelWriter overwrites the previous version of the file, we have to re-insert the remaining sheets as well
##### This step is an intermediate step. After executing this step, our file's "long format" sheet will be ready for tableau viz

In [39]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_fin_data_long_format.to_excel(writer, sheet_name='long format', index=False)
df_read_all_sheets[read_sheet].to_excel(writer, sheet_name=read_sheet)
df_read_all_sheets['Series - Metadata'].to_excel(writer, sheet_name='Series - Metadata')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

#### 4. Segregating the financial inclusion indicators into different column headers
##### The World Bank's data has all the indicators listed in a single column, called indicators
##### This makes it difficult to focus on any single indicator for analysis or visualization
##### So, in this step, we'll seggregate the indicators (and their values) into columns 

#####  4a. Group the data based on different indicators 

In [40]:
grouped = df_fin_data_long_format.groupby(df_fin_data_long_format.code) 

##### 4b. Formatting each group and prepping for the final processing step
##### A lot of pre-processing is happening in this step!
##### We will re-label the 'value' column with the indicator name. Look at how we have done that!


In [41]:
lst_indicator = []
for name, group in grouped:
    indicator_name = group.iloc[0]['indicator'] # we need to rename the column "value" to the relevant indicator name
    group.rename(columns={"value":indicator_name},inplace=True) # How did we do this? We extracted the value of any indicator column of each group (using iloc) to re-label "value" column
    group.drop(columns=['code','indicator'],inplace=True) 
    lst_indicator.append(group) # We are collecting all the grouped dataframes into a list of indicator dataframes

/Users/anshulsaxena/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/anshulsaxena/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### 5. Final processing step
##### We will now combine all the dataframes collected in the list of dataframes above
##### to finally have all the indicator columns and their values in a single dataframe

In [42]:
df_final_processed_data = reduce(lambda df1,df2: pd.merge(df1,df2,on=['country','year']), lst_indicator[:])


#### 6. Writing back to the file

In [43]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_fin_data_long_format.to_excel(writer, sheet_name='long format', index=False)
df_final_processed_data.to_excel(writer, sheet_name='final processed data', index=False)
df_read_all_sheets[read_sheet].to_excel(writer, sheet_name=read_sheet)
df_read_all_sheets['Series - Metadata'].to_excel(writer, sheet_name='Series - Metadata')

# Close the Pandas Excel writer and output the Excel file.
writer.save()